In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate()

Activating environment at `~/Documents/study-validated-numerics/notebook/Project.toml`


# Chapter 2. Interval Arithmetic
## 2.4. Floating Point Interval Arithmetic
### 2.4.1. A ~~MATLAB~~ Julia Implementation of Interval Arithmetic

In [2]:
using FastRounding
using Printf

const SysFloat = Union{Float32, Float64} # same as FastRounding

Union{Float32, Float64}

In [3]:
struct Interval{T<:SysFloat}
    lo::T
    hi::T
    function Interval(lo::T, hi::T) where T<:SysFloat
        lo <= hi ? new{T}(lo, hi) : throw(ArgumentError("The endpoints do not define an interval."))
    end
end

Interval(lo::Real, hi::Real) = Interval(Float64(lo, RoundDown), Float64(hi, RoundUp))
Interval(x::Real) = Interval(x, x)

function Base.show(io::IO, a::Interval{T}) where T<:SysFloat
    print(io, "Interval{$(T)}(", @sprintf("%17.17f", a.lo), ", ", @sprintf("%17.17f", a.hi), ")")
end

In [4]:
a = Interval(3, 4)

Interval{Float64}(3.00000000000000000, 4.00000000000000000)

In [5]:
b = Interval(2, 5)

Interval{Float64}(2.00000000000000000, 5.00000000000000000)

In [6]:
c = Interval(1)

Interval{Float64}(1.00000000000000000, 1.00000000000000000)

In [7]:
Interval(3.0f0, 4.0f0)

Interval{Float32}(3.00000000000000000, 4.00000000000000000)

In [8]:
Interval(1//10)

Interval{Float64}(0.09999999999999999, 0.10000000000000001)

In [9]:
Interval(π)

Interval{Float64}(3.14159265358979312, 3.14159265358979356)

In [10]:
# NG
# Interval(2.0, 1.0)

Define sum of two intervals.

In [11]:
function Base.:+(a::Interval, b::Interval)
    lo = add_round(a.lo, b.lo, RoundDown)
    hi = add_round(a.hi, b.hi, RoundUp)
    Interval(lo, hi)
end

Base.:+(a::Real, b::Interval) = Interval(a) + b
Base.:+(a::Interval, b::Real) = a + Interval(b)

Base.:+(a::Interval) = a

In [12]:
Interval(1, 2) + Interval(3, 4)

Interval{Float64}(4.00000000000000000, 6.00000000000000000)

In [13]:
Interval(1, 2) + Interval(3, 3)

Interval{Float64}(4.00000000000000000, 5.00000000000000000)

In [14]:
Interval(1, 2) + 3

Interval{Float64}(4.00000000000000000, 5.00000000000000000)

In [15]:
3 + Interval(1, 2)

Interval{Float64}(4.00000000000000000, 5.00000000000000000)

Define other unary and binary functions

In [16]:
function Base.:-(a::Interval, b::Interval)
    lo = sub_round(a.lo, b.hi, RoundDown)
    hi = sub_round(a.hi, b.lo, RoundUp)
    Interval(lo, hi)
end

Base.:-(a::Real, b::Interval) = Interval(a) - b
Base.:-(a::Interval, b::Real) = a - Interval(b)

Base.:-(a::Interval) = 0 - a


function Base.:*(a::Interval, b::Interval)
    lo = min(mul_round(a.lo, b.lo, RoundDown),
             mul_round(a.lo, b.hi, RoundDown),
             mul_round(a.hi, b.lo, RoundDown),
             mul_round(a.hi, b.hi, RoundDown))
    hi = max(mul_round(a.lo, b.lo, RoundUp),
             mul_round(a.lo, b.hi, RoundUp),
             mul_round(a.hi, b.lo, RoundUp),
             mul_round(a.hi, b.hi, RoundUp))
    Interval(lo, hi)
end

Base.:*(a::Real, b::Interval) = Interval(a) * b
Base.:*(a::Interval, b::Real) = a * Interval(b)


function Base.:/(a::Interval, b::Interval)
    if (b.lo <= 0.0) & (0.0 <= b.hi)
        throw(ArgumentError("Denominator straddles zero."))
    end

    lo = min(div_round(a.lo, b.lo, RoundDown),
             div_round(a.lo, b.hi, RoundDown),
             div_round(a.hi, b.lo, RoundDown),
             div_round(a.hi, b.hi, RoundDown))
    hi = max(div_round(a.lo, b.lo, RoundUp),
             div_round(a.lo, b.hi, RoundUp),
             div_round(a.hi, b.lo, RoundUp),
             div_round(a.hi, b.hi, RoundUp))
    Interval(lo, hi)
end

Base.:/(a::Real, b::Interval) = Interval(a) / b
Base.:/(a::Interval, b::Real) = a / Interval(b)

Unary

In [17]:
+a

Interval{Float64}(3.00000000000000000, 4.00000000000000000)

In [18]:
-a

Interval{Float64}(-4.00000000000000000, -3.00000000000000000)

Binary

In [19]:
a + b

Interval{Float64}(5.00000000000000000, 9.00000000000000000)

In [20]:
a - b

Interval{Float64}(-2.00000000000000000, 2.00000000000000000)

In [21]:
a * b

Interval{Float64}(6.00000000000000000, 20.00000000000000000)

In [22]:
a / b

Interval{Float64}(0.59999999999999998, 2.00000000000000000)

In [23]:
Interval(1/10)

Interval{Float64}(0.10000000000000001, 0.10000000000000001)

In [24]:
Interval(1)/10

Interval{Float64}(0.09999999999999999, 0.10000000000000001)

Sub-distributivity

In [25]:
Interval(1//10)

Interval{Float64}(0.09999999999999999, 0.10000000000000001)

In [26]:
a = Interval(-1, 1)
b = Interval(-1, 0)
c = Interval(3, 4)

Interval{Float64}(3.00000000000000000, 4.00000000000000000)

In [27]:
a * (b + c)

Interval{Float64}(-4.00000000000000000, 4.00000000000000000)

In [28]:
a * b + a * c

Interval{Float64}(-5.00000000000000000, 5.00000000000000000)

Defial equal

In [29]:
import Base.==

==(a::Interval, b::Interval) = a.lo == b.lo && a.hi == b.hi

== (generic function with 156 methods)

In [30]:
a = Interval(1, 10)
b = Interval(-2, 3)
c = Interval(3, 5)

a == a

true

In [31]:
a == b

false

In [32]:
a == c

false

In [33]:
a != a

false

In [34]:
a != b

true

In [35]:
a != c

true

Inclusion

In [36]:
Base.issubset(a::Interval, b::Interval) = b.lo <= a.lo && a.hi <= b.hi
Base.:⊊(a::Interval, b::Interval) = a ⊆ b && a != b

In [37]:
a ⊆ a

true

In [38]:
a ⊆ b

false

In [39]:
b ⊆ a

false

In [40]:
a ⊊ c

false

In [41]:
c ⊊ a

true

In [42]:
a ⊊ a

false

In [43]:
a ⊇ a

true

In [44]:
a ⊇ c

true

In [45]:
a ⊋ a

false

In [46]:
a ⊋ c

true

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*